# Dog Food Prediction

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.

In [1]:
#################### Initialize ###########################

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('churn').getOrCreate()

# Feature Engineering
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

# Model
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

# Pipeline
from pyspark.ml import Pipeline

# Evaluators
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [2]:
#################### Data ##################################

# Bring in Data
data = spark.read.csv('dog_food.csv',inferSchema=True,header=True)
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [3]:
data.show(5)

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
+---+---+----+---+-------+
only showing top 5 rows



In [4]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [15]:
assembler = VectorAssembler(
  inputCols=['A', 'B', 'C', 'D'],
              outputCol="features")

In [16]:
# Vectorize the data
output = assembler.transform(data)
final_data = output.select('features','Spoiled')

In [17]:
###### Build the Model #################
rfc = RandomForestClassifier(labelCol='Spoiled',featuresCol='features')

# Train model.  This also runs the indexers.
model = rfc.fit(final_data)

In [18]:
# Show feature importance
model.featureImportances

SparseVector(4, {0: 0.017, 1: 0.026, 2: 0.9322, 3: 0.0248})

It looks like C is the most important. Remember the goal was to determine which feature was the most important. 